<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lesson 6.02 - Bootstrapping and Bagging

## Importing libraries


We'll need the following libaries for today's lecture:
- `pandas`
- `numpy`
- `DecisionTreeClassifier` from `sklearn`'s `tree` module
- `BaggingClassifier` from `sklearn`'s `ensemble` module
- `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module
- `accuracy_score` from `sklearn`'s `metrics` module

In [81]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

## Load the Data

We'll be using the `Heart.csv` from the [ISLR Website](https://www-bcf.usc.edu/~gareth/ISL/). There's a copy in this repo under `./datasets/Heart.csv`.

In [59]:
# Read in the Heart .csv data.
df = pd.read_csv('datasets/Heart.csv')

In [60]:
## EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 15 columns):
Unnamed: 0    303 non-null int64
Age           303 non-null int64
Sex           303 non-null int64
ChestPain     303 non-null object
RestBP        303 non-null int64
Chol          303 non-null int64
Fbs           303 non-null int64
RestECG       303 non-null int64
MaxHR         303 non-null int64
ExAng         303 non-null int64
Oldpeak       303 non-null float64
Slope         303 non-null int64
Ca            299 non-null float64
Thal          301 non-null object
AHD           303 non-null object
dtypes: float64(2), int64(10), object(3)
memory usage: 35.6+ KB


In [61]:
df.describe()

,Unnamed: 0,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,299.000000
mean,152.000000,54.438944,0.679868,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.672241
std,87.612784,9.038662,0.467299,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.937438
min,1.000000,29.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000
25%,76.500000,48.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000
50%,152.000000,56.000000,1.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000
75%,227.500000,61.000000,1.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000
max,303.000000,77.000000,1.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000


In [62]:
df.isnull().sum()

Unnamed: 0    0
Age           0
Sex           0
ChestPain     0
RestBP        0
Chol          0
Fbs           0
RestECG       0
MaxHR         0
ExAng         0
Oldpeak       0
Slope         0
Ca            4
Thal          2
AHD           0
dtype: int64

In [63]:
# Drop the `Unnamed: 0` column.
del df['Unnamed: 0']

In [64]:
# Check the first few rows to make sure we dropped the column properly.
df.head(2)

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes


## Data cleaning: Drop rows with null values

In [65]:
# Check the shape of the data.
df.shape

(303, 14)

In [66]:
# How much missing data do we have?


In [67]:
# Drop NAs.
df.dropna(inplace = True)

In [68]:
# Confirm all missing data is dropped.
df.isnull().sum()

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           0
Thal         0
AHD          0
dtype: int64

In [69]:
# What's the shape of our data now?
df.shape

(297, 14)

## Feature Engineering

In [70]:
# Create dummies for the `ChestPain`, `Thal`, and `AHD` columns.
# Be sure to set `drop_first=True`.
df = pd.get_dummies(df,
                   columns=['ChestPain', 'Thal', 'AHD'],
                   drop_first=True)

# Confirm we did this correctly.
df.head()

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_normal,Thal_reversable,AHD_Yes
0,63,1,145,233,1,2,150,0,2.3,3,0.0,0,0,1,0,0,0
1,67,1,160,286,0,2,108,1,1.5,2,3.0,0,0,0,1,0,1
2,67,1,120,229,0,2,129,1,2.6,2,2.0,0,0,0,0,1,1
3,37,1,130,250,0,0,187,0,3.5,3,0.0,1,0,0,1,0,0
4,41,0,130,204,0,2,172,0,1.4,1,0.0,0,1,0,1,0,0


## Model Prep: Create `X` and `y` variables

Our target column will be `AHD_Yes`: 
- 1 means the patient has heart disease
- 0 means they aren't diagnosed with heart disease

In [71]:
X = df.drop('AHD_Yes', axis='columns')

y = df['AHD_Yes']

In [72]:
# What is the accuracy of our baseline model?
y.value_counts(normalize=True)

0    0.538721
1    0.461279
Name: AHD_Yes, dtype: float64

In [73]:
y.mean()

0.4612794612794613

<details><summary>What does a false positive mean in this case?</summary>
    
- A false positive indicates someone **falsely** predict as being in the **positive** class.
- This is someone we incorrectly think has heart disease.
- Incorrectly predicting someone to have heart disease is bad... but it _might_ be worse to incorrectly predict that someone is healthy!
</details>

## Model Prep: Train/Test Split

In [74]:
# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    stratify=y)

## Model Instantiation

In [75]:
# Instantiate `DecisionTreeClassifier` object.
tree = DecisionTreeClassifier(random_state=42)

## Model Evaluation

In [76]:
# Get a cross_val_score for our tree.
cross_val_score(tree, X_train, y_train).mean()

0.6801801801801802

In [77]:
# Fit and score on the training data.
tree.fit(X_train, y_train)
tree.score(X_train, y_train)

1.0

In [78]:
# Score on the testing data.
tree.score(X_test, y_test)

0.72

In [82]:
X_train.columns

Index(['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'RestECG', 'MaxHR', 'ExAng',
       'Oldpeak', 'Slope', 'Ca', 'ChestPain_nonanginal',
       'ChestPain_nontypical', 'ChestPain_typical', 'Thal_normal',
       'Thal_reversable'],
      dtype='object')

In [79]:
tree.feature_importances_

array([0.13813889, 0.02328804, 0.1063808 , 0.089778  , 0.        ,
       0.01360294, 0.03778595, 0.01330065, 0.0940519 , 0.        ,
       0.12694815, 0.0594665 , 0.        , 0.0032388 , 0.25161516,
       0.04240423])

<details><summary> Where do decision trees tend to fall on the Bias/Variance spectrum?</summary>
    
- Decision trees very easily overfit.
- They tend to suffer from **high error due to variance**.
</details>

## Introduction to Ensemble Methods
We can list out the different types of models we've built thus far:
- Linear Regression
- Logistic Regression
- $k$-Nearest Neighbors
- Naive Bayes Classification _(maybe)_

If we want to use any of these models, we follow the same type of process.
1. Based on our problem, we identify which model to use. (Is our problem classification or regression? Do we want an interpretable model?)
2. Fit the model using the training data.
3. Use the fit model to generate predictions.
4. Evaluate our model's performance and, if necessary, return to step 2 and make changes.

So far, we've always had exactly one model. Today, however, we're going to talk about **ensemble methods**. Mentally, you should think about this as if we build multiple models and then aggregate their results in some way.

## Why would we build an "ensemble model?"

Our goal is to estimate $f$, the true function. (Think about $f$ as the **true process** that dictates Ames housing prices.)

We can come up with different hypotheses $h_1$, $h_2$, and so on to get as close to $f$ as possible. (Think about $h_1$ as the model you built to predict $f$, think of $h_2$ as the model your neighbor built to predict $f$, and so on.)

![](./assets/Ensemble.png)


### (Advanced) Three Benefits: Statistical, Computational, Representational
- The **statistical** benefit to ensemble methods: By building one model, our predictions are almost certainly going to be wrong. Predictions from one model might overestimate housing prices; predictions from another model might underestimate housing prices. By "averaging" predictions from multiple models, we'll see that we can often cancel our errors out and get closer to the true function $f$.
- The **computational** benefit to ensemble methods: It might be impossible to develop one model that globally optimizes our objective function. (Remember that CART reach locally-optimal solutions that aren't guaranteed to be the globally-optimal solution.) In these cases, it may be **impossible** for one CART to arrive at the true function $f$. However, generating many different models and averaging their predictions may allow us to get results that are closer to the global optimum than any individual model.
- The **representational** benefit to ensemble methods: Even if we had all the data and all the computer power in the world, it might be impossible for one model to **exactly** equal $f$. For example, a linear regression model can never model a relationship where a one-unit change in $X$ is associated with some *different* change in $Y$ based on the value of $X$. All models have some shortcomings. (See [the no free lunch theorems](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization).) While individual models have shortcomings, by creating multiple models and aggregating their predictions, we can actually create predictions that represent something that one model cannot ever represent.

We can summarize this as the **wisdom of the crowd**.

## Wisdom of the Crowd: Guess the weight of Penelope

![](./assets/penelope.jpg)

[Image source: https://www.npr.org](https://www.npr.org/sections/money/2015/07/17/422881071/how-much-does-this-cow-weigh)

## Ensemble predictions

Let's mimic the "wisdom of the crowd" by creating several decision trees and averaging their predictions on the test set.

In [30]:
# What is this line doing?
predictions = pd.DataFrame(index=X_test.index)

In [32]:
predictions.head()

""
113
195
64
27
245


In [33]:
# Generate ten decision trees.
for i in range(10):
    # Instantiate decision trees.
    t = DecisionTreeClassifier()
    # Fit to our training data.
    t.fit(X_train, y_train)
    # Put predictions in dataframe.
    predictions[f'Tree {i}'] = t.predict(X_test)

predictions

,Tree 0,Tree 1,Tree 2,Tree 3,Tree 4,Tree 5,Tree 6,Tree 7,Tree 8,Tree 9
113,1,1,1,1,1,1,1,1,1,1
195,1,1,1,1,1,1,1,1,1,1
64,1,1,1,1,1,1,1,1,1,1
27,0,0,0,0,0,0,0,0,0,0
245,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
93,1,1,1,1,1,1,1,1,1,1
133,0,0,0,0,0,0,0,0,0,0
33,0,0,0,0,0,0,0,0,0,0
20,1,0,1,0,0,0,1,0,1,1


In [34]:
# Generate aggregated predicted probabilities.
probs = predictions.mean(axis='columns')

In [38]:
# Check out probs.
print([x for x in probs])

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.2, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.8, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.8, 0.4, 0.5, 1.0, 0.0, 0.4, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 1.0]


In [39]:
accuracy_score(y_test, (probs > .5).astype(int))

0.6933333333333334

## Ensemble evaluation?

<details><summary>Why didn't our score improve?</summary>

- Because we fit the same model on the same data ten times!
</details>

## Bootstrapping

Let's get started actually making ensemble predictions. However, in order to do that, we'll need to introduce the idea of bootstrapping, or **random sampling with replacement.**

### Summary
When bootstrapping in order to fit multiple estimators, we want to:
- Take a sample of size $n$.
- With replacement.
- From our original data.

<details><summary>Why do you think we want to take a sample of size n?</summary>
    
- Because we want our estimators to be fit on data of the same size!
- If our original data had $n = 1,000$ and we bootstrapped a sample of size 50 to fit an estimator on, our estimator fit on size 50 will probably look very, very different from an estimator fit on size 1,000.
</details>

<details><summary>Why do you think we want to sample with replacement?</summary>
    
- If we didn't sample with replacement, we'd just get identical samples of size $n$. (These would be copies of our original data!)
</details>

Boostrapping with `pandas`:

In [41]:
X_train.shape

(222, 16)

In [42]:
X_train.shape[0]

222

In [40]:
X_train.sample(n=X_train.shape[0],
               replace=True,
               random_state=42)

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_normal,Thal_reversable
59,51,1,125,213,0,2,125,1,1.4,1,1.0,0,0,1,1,0
234,54,0,160,201,0,0,163,0,0.0,1,1.0,1,0,0,1,0
50,41,0,105,198,0,0,168,0,0.0,1,1.0,0,1,0,1,0
117,35,0,138,183,0,0,182,0,1.4,1,0.0,0,0,0,1,0
10,57,1,140,192,0,0,148,0,0.4,2,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,45,1,104,208,0,2,148,1,3.0,2,0.0,0,0,0,1,0
28,43,1,150,247,0,0,171,0,1.5,1,0.0,0,0,0,1,0
91,62,0,160,164,0,2,145,0,6.2,3,3.0,0,0,0,0,1
140,59,1,140,221,0,0,164,1,0.0,1,0.0,0,1,0,1,0


## Bagging: Bootstrap Aggregating

As we have seen, decision trees are very powerful machine learning models. However, decision trees have some limitations. In particular, trees that are grown very deep tend to learn highly irregular patterns (a.k.a. they overfit their training sets). 

Bagging (bootstrap aggregating) mitigates this problem by exposing different trees to different sub-samples of the training set.

The process for creating bagged decision trees is as follows:
1. From the original data of size $n$, bootstrap $k$ samples each of size $n$ (with replacement!).
2. Build a decision tree on each bootstrapped sample.
3. Make predictions by passing a test observation through all $k$ trees and developing one aggregate prediction for that observation.

![](./assets/Ensemble.png)

### What do you mean by "aggregate prediction?"
As with all of our modeling techniques, we want to make sure that we can come up with one final prediction for our observation. (Building 1,000 trees and coming up with 1,000 predictions for one observation probably wouldn't be very helpful.)

Suppose we want to predict whether or not a Reddit post is going to go viral, where `1` indicates viral and `0` indicates non-viral. We build 100 decision trees. Given a new Reddit post labeled `X_test`, we pass these features into all 100 decision trees.
- 70 of the trees predict that the post in `X_test` will go viral.
- 30 of the trees predict that the post in `X_test` will not go viral.

**`.predict_proba(X_test)` to do?**
<details><summary>What might you expect .predict(X_test) do?</summary>

- `.predict(X_test)` should output a 1, predicting that the post will go viral.

</details>

<details><summary>What might you expect .predict_proba(X_test) do?</summary>

- `.predict_proba(X_test)` should output 0.7, indicating the probability of the post going viral is 70%.
</details>


## Bagging Classifier using a `for` loop

In the cell below, we'll create an ensemble of trees like before, except this time we'll train each tree to a **bootstrapped** sample of the training data.

In [44]:
# Instantiate dataframe.
predictions = pd.DataFrame(index=X_test.index)

# Generate ten decision trees.
for i in range(10):
    
    # Bootstrap X data.
    # Should we add a random seed?
    X_sample = X_train.sample(n=X_train.shape[0],
                             replace=True)
    
    # Get y data that matches the X data.
    y_sample = y_train[X_sample.index]
    
    # Instantiate decision trees.
    t = DecisionTreeClassifier()
    
    # Fit to our training data.
    t.fit(X_sample, y_sample)
    
    # Put predictions in dataframe.
    predictions[f'Tree {i}'] = t.predict(X_test)

predictions.head(20)

,Tree 0,Tree 1,Tree 2,Tree 3,Tree 4,Tree 5,Tree 6,Tree 7,Tree 8,Tree 9
113,1,0,1,1,1,1,1,1,1,1
195,0,0,1,0,1,0,0,1,1,0
64,1,1,0,1,1,1,1,1,1,1
27,1,0,0,1,0,1,1,0,1,1
245,1,1,0,1,0,1,1,0,0,1
210,0,0,0,0,0,0,0,0,0,0
221,0,1,0,1,1,0,0,1,0,0
1,1,1,1,1,1,1,1,1,1,1
116,0,0,0,0,0,0,0,0,0,0
157,1,1,1,0,1,1,1,1,1,1


In [45]:
# Generate aggregated predicted probabilities.
predictions.mean(axis='columns')

113    0.9
195    0.4
64     0.9
27     0.6
245    0.6
      ... 
93     0.2
133    0.0
33     0.5
20     0.4
76     1.0
Length: 75, dtype: float64

In [46]:
# Generate aggregated predicted probabilities as probs.
probs = predictions.mean(axis='columns')

In [47]:
accuracy_score(y_test, (probs >= .5).astype(int))

0.84

## Bagging Classifier using `sklearn`

[BaggingClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)

In the cell below, create and score instance of `BaggingClassifier` on the test set. You should get a similar score to the one in the previous step.

In [55]:
# Instantiate BaggingClassifier.
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=42, verbose=1, n_estimators=100)

In [57]:
# Fit BaggingClassifier.
bag.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=100, n_jobs=1, oob_score=False,
         random_state=42, verbose=1, warm_start=False)

In [58]:
# Score BaggingClassifier.
bag.score(X_test, y_test)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


0.84